In [1]:
from sklearn import model_selection
from gensim import models,corpora,similarities
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import TaggedDocument,Doc2Vec
from gensim.models import LdaModel
import numpy as np
import os
import random
from sklearn.neighbors import KNeighborsClassifier 
from gensim.models import word2vec
import sklearn.metrics as metrics


/home/jovyan/.rsm-msba/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
dropbox_folder = None

for dirname, dirnames, filenames in os.walk(os.path.expanduser('~')):
    for subdirname in dirnames:
        if(subdirname == 'Dropbox'):
            dropbox_folder = os.path.join(dirname, subdirname)
            break
    if dropbox_folder:
        break

with open(dropbox_folder + 'data.txt', 'r') as file:
    doc = [c.split(',') for c in file.read().split('\n')]

In [3]:
# train word2vec model
size_word=100
size_lda=12
print("train word2vec model")
model_wv=word2vec.Word2Vec(doc, size=size_word,workers=2,min_count=1,iter=10)
dictionary = corpora.Dictionary(doc)
corpus = [ dictionary.doc2bow(text) for text in doc ]

train word2vec model


In [4]:
#train lda model
print("train lda model")
lda = LdaModel(corpus=corpus,id2word=dictionary, num_topics=size_lda)

train lda model


In [5]:
#lda.get_document_topics
for doc_num in doc:
    doc_bow=dictionary.doc2bow(doc_num)
    doc_lda = lda[doc_bow]
    print ("the topic distribution of the new document",doc_lda)
    break
for topic in doc_lda:
    print ("Proportion of the topic","%s\t%f\n"%(lda.print_topic(topic[0]), topic[1]))

the topic distribution of the new document [(4, 0.7591665), (7, 0.22309174)]
Proportion of the topic 0.034*"number" + 0.032*"card" + 0.015*"email" + 0.015*"paper" + 0.015*"information" + 0.012*"address" + 0.010*"date" + 0.010*"account" + 0.009*"credit" + 0.007*"read"	0.759166

Proportion of the topic 0.030*"glasses" + 0.009*"horizon" + 0.009*"connected" + 0.008*"send" + 0.008*"iphone" + 0.008*"connect" + 0.007*"power" + 0.007*"verizon" + 0.007*"button" + 0.007*"device"	0.223092



In [6]:
print (lda.get_topic_terms(0, topn=10)) 

[(82, 0.014549933), (705, 0.014494144), (49, 0.012492223), (164, 0.0117205605), (0, 0.008091554), (80, 0.007492071), (325, 0.0074825534), (90, 0.006868124), (950, 0.0066822777), (39, 0.0066119293)]


In [7]:
# get the main word in each topic
aa=[x[0] for x in lda.get_topic_terms(0, topn=10)]
for it in aa:
    print ("NO.%d topic NO.%d main word"%(0,it), dictionary[it])

print  (lda.get_term_topics(0, minimum_probability=0.05))

print (lda.get_document_topics(corpus[0], minimum_probability=None, minimum_phi_value=0.1, per_word_topics=False))

NO.0 topic NO.82 main word password
NO.0 topic NO.705 main word teamviewer
NO.0 topic NO.49 main word computer
NO.0 topic NO.164 main word email
NO.0 topic NO.0 main word address
NO.0 topic NO.80 main word page
NO.0 topic NO.325 main word minutes
NO.0 topic NO.90 main word read
NO.0 topic NO.950 main word google
NO.0 topic NO.39 main word account
[]
[(4, 0.7592648), (7, 0.22299334)]


In [8]:
print ("Mapping the topic to the word2vec space")
the_id=[]
the_vl=[]
the_w =[]

Mapping the topic to the word2vec space


In [9]:
print ("Calculate the weight of each word in the topic")
for x in range(size_lda):
    the_id.append([xx[0] for xx in lda.get_topic_terms(x,topn=5)])
    the_sum=sum([xx[1] for xx in lda.get_topic_terms(x,topn=5)])
    the_w.append([xx[1]/the_sum for xx in lda.get_topic_terms(x,topn=5)])  

Calculate the weight of each word in the topic


In [10]:
# Mapping the topic to the word2vec space
print (" mapping to coordinates")
m=0
the_wv=np.zeros([size_lda,size_word])

for it in the_id:
    n=0
    for it_id in it:
        word_t=dictionary[it_id]
        #print (word_t+"**",np.shape(model_wv[word_t]),the_w[m][n])
        the_wv[m]+=[x_word*the_w[m][n] for x_word in model_wv[word_t]]
        n+=1
    m+=1
doc_word=np.zeros([len(doc),size_word])

 mapping to coordinates


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [11]:
# Mapping the documents to the word2vec space
print ("Mapping the document to word2vec")
m=0
for each_doc in doc:
    for each_word in each_doc:
        #print each_word
        doc_word[m]+=model_wv[each_word]
    n=0
    for doc_word_each in doc_word[m]:
        doc_word[m][n]=doc_word[m][n]/len(doc_word[m])
        n+=1
    m+=1

Mapping the document to word2vec


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [12]:
# calculate the distance bewteen the each document to each topic
print ("calculate the distance bewteen the document to topic")
def destince(a,b):
    dt=0
    for each_dt in range(len(a)):
        dt+=(a[each_dt]-b[each_dt])*(a[each_dt]-b[each_dt])
    return np.sqrt(dt)
doc_t=np.zeros([len(doc_word),size_lda])
m=0
for each_d in doc_word:
    n=0
    for each_t in the_wv:
        doc_t[m][n]=destince(each_d,each_t)
        n+=1
    m+=1


calculate the distance bewteen the document to topic


In [13]:
#make the nearest topic the domain topic
topic=[]
for i in doc_t:
    topic.append(np.argmin(i))

In [14]:
##calculate the distribution of the topic
import collections
counter=collections.Counter(topic)
print(counter)

Counter({6: 4922, 1: 1719, 10: 481, 8: 33, 3: 25, 5: 25, 9: 25, 2: 21, 11: 8, 7: 7, 4: 1})
